In [233]:
import os

In [234]:
%pwd

'd:\\Python_DataScience\\Wine_Quality_Prediction\\Wine_Quality_Prediction_End_To_End_ML'

In [235]:
%pwd

'd:\\Python_DataScience\\Wine_Quality_Prediction\\Wine_Quality_Prediction_End_To_End_ML'

In [236]:
from dataclasses import dataclass
from pathlib import Path

@dataclass
class ModelEvaluationConfig:
    root_dir: Path
    model_path: Path
    train_data_path: Path
    test_data_path: Path
    train_metrics_file: str
    test_metrics_file: str
    name: str
    model_name: str

In [237]:
from wine_pred.constants import *

from wine_pred.utils.common import read_yaml, create_directories

In [238]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 schema_filepath = SCHEMA_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        schema = self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir = config.root_dir,
            model_path = config.model_path,
            train_data_path = config.train_data_path,
            test_data_path=config.test_data_path,
            train_metrics_file=config.train_metrics_file,
            test_metrics_file=config.test_metrics_file,
            name = schema.name,
            model_name=config.model_name
        )

        return model_evaluation_config

In [239]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
import pandas as pd
from wine_pred.constants import *
import numpy as  np
from wine_pred.logging.logger import logger
from wine_pred.utils.common import load_bin, save_json
import joblib
import json

In [240]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    def metrics(self):

        train_df = pd.read_csv(self.config.train_data_path)
        test_df = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)
        target_column = self.config.name

        X_train = train_df.drop([target_column], axis=1)
        X_test = test_df.drop([target_column],axis=1)
        y_train = train_df[target_column]
        y_test = test_df[target_column]

        y_train_pred = model.predict(X_train)
        y_test_pred = model.predict(X_test)

        train_json = {
            'RMSE': np.sqrt(mean_squared_error(y_train,y_train_pred)),
            'MAE': mean_absolute_error(y_train,y_train_pred),
            'R2 Score': r2_score(y_train,y_train_pred)
        }

        test_json = {
            'RMSE': np.sqrt(mean_squared_error(y_test,y_test_pred)),
            'MAE': mean_absolute_error(y_test,y_test_pred),
            'R2 Score': r2_score(y_test,y_test_pred)
        }

        train_path = 'artifacts/model_evaluation/train_mertics.json'
        test_path = 'artifacts/model_evaluation/test_mertics.json'

        with open(train_path, 'w') as f:
            json.dump(train_json, f)

        with open(test_path, 'w') as f:
            json.dump(test_json, f)

        # save_json(train_path,train_json)
        # save_json(test_path,test_json,)

        logger.info('Train & Test Metrics Json file saved sucessfully.')
       
        

In [241]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation = ModelEvaluation(model_evaluation_config)
    model_evaluation.metrics()
    
except Exception as e:
    logger.error(f"An error occurred: {e}")
    raise e

[ 2024-07-08 16:39:56,890 ] 29 root - INFO - yaml file: config\config.yaml loaded succesfully
[ 2024-07-08 16:39:56,891 ] 29 root - INFO - yaml file: schema.yaml loaded succesfully
[ 2024-07-08 16:39:56,891 ] 43 root - INFO - Created directory at: artifacts
[ 2024-07-08 16:39:56,891 ] 43 root - INFO - Created directory at: artifacts/model_evaluation
[ 2024-07-08 16:39:56,925 ] 44 root - INFO - Train & Test Metrics Json file saved sucessfully.
